In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

In [27]:
headers = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Origin': 'https://index.hu',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://index.hu/24ora/?s=&tol=1999-01-01&ig=2019-12-13&profil=&rovat=&cimke=&word=1&pepe=1',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en;q=0.9,hu-HU;q=0.8,hu;q=0.7,en-US;q=0.6',
}

data = {
  'datum': '2019-12-20',
  'rovat': '24ora/',
  'url_params[ig]': '2019-12-20',
  'url_params[pepe]': '1',
  'url_params[rovat]': '',
  'url_params[s]': '',
  'url_params[tol]': '1999-01-01',
  'url_params[alllowRovatChoose]': '0',
  'url_params[profil]': '',
  'url_params[cimke]': '',
  'url_params[word]': '1',
  'url_params[p]': '1'
}

In [28]:
dict_list = []
for page in tqdm(range(30)):
    data["url_params[p]"] = str(page)
    response = requests.post('https://index.hu/api/json/', headers=headers, data = data)
    dict_list.append(response.json()['list'])

In [29]:
index_dict = {}
for dik in tqdm(dict_list):
    related_urls = []
    url = dik[0]["url"].replace("//", "https://")
    # print(url)
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    cikktorzs = soup.find('div', class_='cikk-torzs')
    if cikktorzs is not None:
        if cikktorzs.find_all("p") is not None:
            for paragraph in cikktorzs.find_all("p"):
                if paragraph.find_all("a") is not None:
                    for link in paragraph.find_all("a"):
                        if "http" in link.get('href'):
                            related_urls.append(link.get('href'))
                        else:
                            related_urls.append("https:" + link.get('href'))
    index_dict[url] = {
        "related_links" : related_urls,
        "publish_date" : dik[0]["datum"],
        "last_modification_date" : dik[0]["_mod_ts"]
    }

In [30]:
index_dict

{'https://index.hu/kultur/2019/12/19/boszorkany_fesztival_miss_bubaj_szepsegverseny_aura-csakra_vizsgalat_magus_varazslo/': {'related_links': ['https://index.hu/kultur/2019/11/21/kiskunfelegyhaza_piramis_diszko_magyar_boszorkanyszovetseg_piramis_viz_binger_vilmos/'],
  'publish_date': '2 órája',
  'last_modification_date': '2019-12-19 17:34:28'},
 'https://index.hu/kultur/2019/12/19/rendszervaltas_30_euforia_capa_kozpont_foto_kiallitas/': {'related_links': ['https://capacenter.hu/kiallitasok/euforia/',
   'https://index.hu/belfold/tavkony0202/',
   'https://index.hu/kultur/2019/07/27/kiraly_tamas_out_the_box_ludwig_muzeum_kiallitas_divattervezo/'],
  'publish_date': '9 órája',
  'last_modification_date': '2019-12-19 10:33:24'},
 'https://index.hu/techtud/2019/12/18/mi_lesz_meg_a_2020-ra_igert_technologiakbol/': {'related_links': ['https://index.hu/tech/2010/05/05/atomot_a_mobilokba/',
   'https://index.hu/tudomany/2015/01/16/elon_musk_is_kolonizalna_a_marsot/',
   'https://index.hu/tud